In [3]:
import pandas as pd
import pickle
from langchain_community.vectorstores import chroma as Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_community.document_loaders import DataFrameLoader
from langchain.indexes import SQLRecordManager, index
from langchain_core.documents import Document

In [4]:
##FSD_1555
dataPath = "/home/mbhatti/mnt/d/LLM-repo1/models/langchain_implementation/fsd_1555_0601_21_59_22TO23_59_59.pkl"
dateFrom = "2023-06-01 22:59:45+00:00" 
dateTo = "2023-06-01 23:59:59+00:00" #200 tweets labelled

In [5]:
# Loading pandas dataframe from picke file
with open(dataPath, 'rb') as f:
    data = pickle.load(f)

df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'])
# df = df.drop(columns=['id','tag_class', 'source', 'lang', 'urls','locations'])

#Get data between thresholds
threshold_datetime_lower = pd.to_datetime(dateFrom)
threshold_datetime_upper = pd.to_datetime(dateTo)
df = df[df['date'] >= threshold_datetime_lower]
df = df[df['date'] <= threshold_datetime_upper]
df_new  = df.drop_duplicates(subset=["text"], keep=False)
#Covert date to string
df['date'] = df['date'].astype(str)
df

,date,text
7297,2023-06-01 23:59:59+00:00,◆ Heavy Rain and Flood Warning in Toyohashi Ci...
7298,2023-06-01 23:57:15+00:00,"At 6:38 a.m., a large rainstorm warning was is..."
7299,2023-06-01 23:57:06+00:00,"Everyone, are you okay with the effects of typ..."
7300,2023-06-01 23:56:42+00:00,Good morning #It's raining in Hikone City I've...
7301,2023-06-01 23:56:39+00:00,Osaka City has a temperature of 22.2°C humidit...
...,...,...
7491,2023-06-01 23:01:14+00:00,[Climate Warning] 2023/6/2 6:11 Hikone Regiona...
7492,2023-06-01 23:00:51+00:00,"June 02, 2023 (Friday) 08:00Currently #Hurry R..."
7493,2023-06-01 23:00:44+00:00,Situation of Dams and Inflows under Flood Cont...
7494,2023-06-01 23:00:15+00:00,A flood warning was issued in southern Hamamat...


In [6]:
def bgeEmbeddings():
    model_name = "BAAI/bge-large-en-v1.5"
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
    model = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    return model

In [20]:
embeddings = bgeEmbeddings()


documents = []
loader = DataFrameLoader(data, page_content_column="text")
documents.extend(loader.load())
#Create a vector store
# db = Chroma.Chroma("Langchain collection",embeddings)
db = Chroma.Chroma.from_documents(documents,embeddings)
if db._client.list_collections() != None:

  for collection in db._client.list_collections():
    ids = collection.get()['ids']
    print('REMOVE %s document(s) from %s collection' % (str(len(ids)), collection.name))
    if len(ids): collection.delete(ids)

db = Chroma.Chroma.from_documents(documents,embeddings)

REMOVE 800 document(s) from langchain collection


In [21]:
# Get all embeddings
len(db._collection.get()['ids'])

400

In [8]:
x = x['ids']


In [1]:
if db._client.list_collections() != None:

  for collection in db._client.list_collections():
    ids = collection.get()['ids']
    print('REMOVE %s document(s) from %s collection' % (str(len(ids)), collection.name))
    if len(ids): collection.delete(ids)

NameError: name 'db' is not defined

In [8]:
print(db._collection.get())

{'ids': ['1947deae-ec80-11ee-b980-78ac4456dba0', '1947df44-ec80-11ee-b980-78ac4456dba0', '1947df76-ec80-11ee-b980-78ac4456dba0', '1947df9e-ec80-11ee-b980-78ac4456dba0', '1947dfbc-ec80-11ee-b980-78ac4456dba0', '1947dfe4-ec80-11ee-b980-78ac4456dba0', '1947e002-ec80-11ee-b980-78ac4456dba0', '1947e03e-ec80-11ee-b980-78ac4456dba0', '1947e066-ec80-11ee-b980-78ac4456dba0', '1947e084-ec80-11ee-b980-78ac4456dba0', '1947e0a2-ec80-11ee-b980-78ac4456dba0', '1947e0c0-ec80-11ee-b980-78ac4456dba0', '1947e0de-ec80-11ee-b980-78ac4456dba0', '1947e0fc-ec80-11ee-b980-78ac4456dba0', '1947e110-ec80-11ee-b980-78ac4456dba0', '1947e12e-ec80-11ee-b980-78ac4456dba0', '1947e14c-ec80-11ee-b980-78ac4456dba0', '1947e16a-ec80-11ee-b980-78ac4456dba0', '1947e188-ec80-11ee-b980-78ac4456dba0', '1947e19c-ec80-11ee-b980-78ac4456dba0', '1947e1ba-ec80-11ee-b980-78ac4456dba0', '1947e1d8-ec80-11ee-b980-78ac4456dba0', '1947e1f6-ec80-11ee-b980-78ac4456dba0', '1947e20a-ec80-11ee-b980-78ac4456dba0', '1947e228-ec80-11ee-b980-78ac44